In [12]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, rank
from pyspark.sql.window import Window

In [13]:
# Create Spark session
spark = SparkSession.builder.appName("PySparkCaseStudy").getOrCreate()

In [14]:
# Load CSV files into DataFrames
employees = spark.read.csv("employees.csv", header=True, inferSchema=True)
departments = spark.read.csv("departments.csv", header=True, inferSchema=True)

In [15]:
# Show loaded data
employees.show()
departments.show()

+-----+-------+------+------+----------+
|EmpID|   Name|DeptID|Salary|  JoinDate|
+-----+-------+------+------+----------+
|    1|  Alice|   101| 70000|2021-01-10|
|    2|    Bob|   102| 48000|2020-03-15|
|    3|Charlie|   101| 52000|2021-07-01|
|    4|  Diana|   103| 82000|2019-11-23|
|    5|    Eve|   102| 46000|2022-05-10|
|    6|  Frank|   101| 62000|2023-07-09|
|    7|    Gia|   102| 74000|2021-03-10|
|    8|  Harry|   102| 68000|2020-09-20|
|    9|    Ian|   101| 57000|2021-10-21|
|   10|  Jenny|   103| 85000|2019-11-12|
+-----+-------+------+------+----------+

+------+-----------+
|DeptID|   DeptName|
+------+-----------+
|   101|Engineering|
|   102|      Sales|
|   103|         HR|
+------+-----------+



1. Filter: Get employees earning more than ₹50,000

In [16]:
high_earners = employees.filter(col("Salary") > 50000)
high_earners.show()

+-----+-------+------+------+----------+
|EmpID|   Name|DeptID|Salary|  JoinDate|
+-----+-------+------+------+----------+
|    1|  Alice|   101| 70000|2021-01-10|
|    3|Charlie|   101| 52000|2021-07-01|
|    4|  Diana|   103| 82000|2019-11-23|
|    6|  Frank|   101| 62000|2023-07-09|
|    7|    Gia|   102| 74000|2021-03-10|
|    8|  Harry|   102| 68000|2020-09-20|
|    9|    Ian|   101| 57000|2021-10-21|
|   10|  Jenny|   103| 85000|2019-11-12|
+-----+-------+------+------+----------+



2. Join: Add department names to the employee table

In [17]:
emp_with_dept = employees.join(departments, on="DeptID", how="inner")
emp_with_dept.show()

+------+-----+-------+------+----------+-----------+
|DeptID|EmpID|   Name|Salary|  JoinDate|   DeptName|
+------+-----+-------+------+----------+-----------+
|   101|    1|  Alice| 70000|2021-01-10|Engineering|
|   102|    2|    Bob| 48000|2020-03-15|      Sales|
|   101|    3|Charlie| 52000|2021-07-01|Engineering|
|   103|    4|  Diana| 82000|2019-11-23|         HR|
|   102|    5|    Eve| 46000|2022-05-10|      Sales|
|   101|    6|  Frank| 62000|2023-07-09|Engineering|
|   102|    7|    Gia| 74000|2021-03-10|      Sales|
|   102|    8|  Harry| 68000|2020-09-20|      Sales|
|   101|    9|    Ian| 57000|2021-10-21|Engineering|
|   103|   10|  Jenny| 85000|2019-11-12|         HR|
+------+-----+-------+------+----------+-----------+



3. GroupBy & Aggregation: Average salary by department

In [18]:
avg_salary = emp_with_dept.groupBy("DeptName").agg(avg("Salary").alias("AvgSalary"))
avg_salary.show()

+-----------+---------+
|   DeptName|AvgSalary|
+-----------+---------+
|      Sales|  59000.0|
|Engineering|  60250.0|
|         HR|  83500.0|
+-----------+---------+



4. Aggregation: Maximum salary in the company

In [19]:
max_salary = employees.agg(max("Salary").alias("MaxSalary"))
max_salary.show()

+---------+
|MaxSalary|
+---------+
|    85000|
+---------+



5. Window Function: Rank employees within department by salary

In [20]:
# Define window spec
window_spec = Window.partitionBy("DeptID").orderBy(col("Salary").desc())

# Add rank
ranked_employees = employees.withColumn("Rank", rank().over(window_spec))
ranked_employees.show()

+-----+-------+------+------+----------+----+
|EmpID|   Name|DeptID|Salary|  JoinDate|Rank|
+-----+-------+------+------+----------+----+
|    1|  Alice|   101| 70000|2021-01-10|   1|
|    6|  Frank|   101| 62000|2023-07-09|   2|
|    9|    Ian|   101| 57000|2021-10-21|   3|
|    3|Charlie|   101| 52000|2021-07-01|   4|
|    7|    Gia|   102| 74000|2021-03-10|   1|
|    8|  Harry|   102| 68000|2020-09-20|   2|
|    2|    Bob|   102| 48000|2020-03-15|   3|
|    5|    Eve|   102| 46000|2022-05-10|   4|
|   10|  Jenny|   103| 85000|2019-11-12|   1|
|    4|  Diana|   103| 82000|2019-11-23|   2|
+-----+-------+------+------+----------+----+



6. Combine Actions: Filter + Join + GroupBy + Aggregation

In [21]:
# Find avg salary of departments that have at least one employee earning >60k
filtered = employees.filter(col("Salary") > 60000)
filtered_with_dept = filtered.join(departments, "DeptID")
result = filtered_with_dept.groupBy("DeptName").agg(avg("Salary").alias("AvgHighSalary"))
result.show()

+-----------+-------------+
|   DeptName|AvgHighSalary|
+-----------+-------------+
|      Sales|      71000.0|
|Engineering|      66000.0|
|         HR|      83500.0|
+-----------+-------------+

